In [16]:
# import the libraries
import pandas as pd
import numpy as np

<h1>Create windows and extract features for training and testing data<h1>

In [17]:
def extract_features(data):
    '''Function to extract features from the data set'''
    size = len(data)
    parts = ['L_heel_','L_ball_','R_heel_','R_ball_']
    rows = []
    for i in range(0,size-30): # window size of 30 seconds overlapping one another
        current = data.iloc[i:i+30,:]
        # calculate features for window
        features = {}
    
        for p in parts:
        
            features[p+'temp_mean'] = current[p+'temp'].mean()
            features[p+'temp_std'] = current[p+'temp'].std()
            features[p+'pressure_mean'] = current[p+'pressure'].mean()
            features[p+'pressure_std'] = current[p+'pressure'].std()
            features[p+'risk'] = current[p+'risk'].mode()[0]
        
        
        features['CrossFootDifference_temp_heel'] = ((current['L_heel_temp'] - current['R_heel_temp']).mean())
        features['CrossFootDifference_temp_ball'] = ((current['L_ball_temp'] - current['R_ball_temp']).mean())
        features['CrossFootDifference_pressure_heel'] = ((current['L_heel_pressure'] - current['R_heel_pressure']).mean())
        features['CrossFootDifference_pressure_ball'] = ((current['L_ball_pressure'] - current['R_ball_pressure']).mean())

        rows.append(features)

    features_df = pd.DataFrame(rows)
    return features_df

In [ ]:
# get the training data and testing data
training_data = pd.read_csv('simulated_data_training.csv')
training_data = extract_features(training_data)
testing_data = pd.read_csv('simulated_data_testing.csv')
testing_data = extract_features(testing_data)